In [ ]:
from AlgorithmImports import *

class CrackSpreadAlgorithm(QCAlgorithm):
    def Initialize(self):
        # Configuración de la estrategia
        self.SetStartDate(2010, 1, 1)  # Fecha de inicio
        self.SetEndDate(2023, 12, 31)  # Fecha de fin
        self.SetCash(25000)  # Capital inicial
        
        # Añadir activos al algoritmo
        self.cl = self.AddFuture(Futures.Energies.CrudeOilWTI).Symbol
        self.rb = self.AddFuture(Futures.Energies.Gasoline).Symbol
        self.ho = self.AddFuture(Futures.Energies.HeatingOil).Symbol
        self.refiner_symbol = self.AddEquity("PSX", Resolution.Daily).Symbol
        
        # Parámetros de la estrategia
        self.window = 21  # Ventana de 21 días
        self.thresh = 2  # Umbral de Z-score
        
        # Inicialización de las rolling windows
        self.rb_window = RollingWindow[float](self.window)
        self.ho_window = RollingWindow[float](self.window)
        self.cl_window = RollingWindow[float](self.window)
        self.refiner_window = RollingWindow[float](self.window)
        
        # Para calcular el Z-score y divergencias
        self.zscore = 0
        self.last_price = 0

    def OnData(self, data):
        # Asegúrate de que todos los datos estén disponibles
        if not (data.ContainsKey(self.cl) and data.ContainsKey(self.rb) and data.ContainsKey(self.ho) and data.ContainsKey(self.refiner_symbol)):
            return
        
        # Agregar datos a las ventanas rodantes
        self.rb_window.Add(data[self.rb].Close)
        self.ho_window.Add(data[self.ho].Close)
        self.cl_window.Add(data[self.cl].Close)
        self.refiner_window.Add(data[self.refiner_symbol].Close)
        
        # Verificar que las ventanas rodantes estén listas
        if self.rb_window.IsReady and self.ho_window.IsReady and self.cl_window.IsReady and self.refiner_window.IsReady:
            # Calcular crackspread por barril
            rb_per_barrel = self.rb_window[0] * 42
            ho_per_barrel = self.ho_window[0] * 42
            cl_price = self.cl_window[0]
            crack_spread = (2 * rb_per_barrel + ho_per_barrel - 3 * cl_price) / 3
            
            # Normalizar precios y calcular el rank
            crack_spread_rank = self.PercentileRank(self.rb_window, crack_spread)
            refiner_rank = self.PercentileRank(self.refiner_window, data[self.refiner_symbol].Close)
            
            # Diferencia de ranks y cálculo de Z-score
            rank_spread = refiner_rank - crack_spread_rank
            mean_spread = np.mean([bar for bar in self.refiner_window])
            std_spread = np.std([bar for bar in self.refiner_window])
            self.zscore = (rank_spread - mean_spread) / std_spread
            
            # Generar señales de compra y venta
            if self.zscore < -self.thresh and not self.Portfolio[self.refiner_symbol].Invested:
                self.SetHoldings(self.refiner_symbol, 1)  # Entra en posición
            elif self.zscore > self.thresh and self.Portfolio[self.refiner_symbol].Invested:
                self.Liquidate(self.refiner_symbol)  # Sal de la posición

    def PercentileRank(self, window, value):
        sorted_window = sorted([bar for bar in window])  # Ordena los datos
        # Encuentra la posición relativa del valor dentro del rango ordenado
        count = sum(1 for x in sorted_window if x <= value)
        percentile_rank = count / len(sorted_window)
        return percentile_rank
